This is a program that returns a portfolio analysis based off the user's inputs. 

User inputs tickers, date range, and frequency (daily, monthly...).

Program outputs summary excel file. Remember to change the file path.

I am currently troubleshooting a bug that is giving an excel error for the expected return and sharpe values. To troubleshoot, edit the formulas and remove any "@" symbol.

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

#WIP-troubleshooting excel errors
def xl_col(col_idx: int) -> str:
    """0-based index -> Excel column letter."""
    col = ""
    x = col_idx + 1
    while x:
        x, r = divmod(x - 1, 26)
        col = chr(65 + r) + col
    return col

def safe_val(x):
    if isinstance(x, (float, int, np.floating, np.integer)):
        return float(x) if np.isfinite(x) else None
    if x is None or (isinstance(x, str) and x == ""):
        return None
    return x
    
def safe_row(seq):
    return [safe_val(v) for v in seq]
#################
#USER INPUTS
tickers = ["AAPL", "BND", "CCI", "EQR", "SOFI", "WSBC", "XOM"]
start_date = "2020-09-01"
end_date   = "2025-09-30"
frequency  = "1mo"   # 1d, 1wk, 1mo, 3mo
rf_annual  = 0.01    # 1%
weights    = None    

out_dir = Path(r"C:\Users\holde\OneDrive\University of Denver\FIN4000\outputs")
out_dir.mkdir(parents=True, exist_ok=True)
excel_path = out_dir / "portfolio_dashboard.xlsx"

data = yf.download(
    tickers=tickers,
    start=start_date,
    end=end_date,
    interval=frequency,
    auto_adjust=False,
    progress=False,
    group_by="ticker"
)

if data is None or data.empty:
    raise RuntimeError("No data downloaded. Check tickers/dates/interval.")


if isinstance(data.columns, pd.MultiIndex):
    fields = data.columns.get_level_values(1)
    if "Adj Close" in fields:
        prices = data.xs("Adj Close", axis=1, level=1).copy()
    elif "Close" in fields:
        prices = data.xs("Close", axis=1, level=1).copy()
    else:
        raise RuntimeError()
else:
    if "Adj Close" in data.columns:
        prices = data[["Adj Close"]].copy()
    elif "Close" in data.columns:
        prices = data[["Close"]].copy()
    else:
        raise RuntimeError("Neither 'Adj Close' nor 'Close' present.")
    prices.columns = [tickers[0]]

# Clean + filter
prices = prices.reindex(columns=[t for t in tickers if t in prices.columns])
prices = prices.clip(lower=0.01, upper=1_000_000)
prices = prices.where((prices > 0) & (prices < 1_000_000))
prices = prices.dropna(axis=0, how="all").dropna(axis=1, how="all")

if prices.empty:
    raise RuntimeError("No price data after cleaning. Adjust tickers/dates/interval.")

# ---------- RETURNS / STATS ----------
returns = prices.pct_change().dropna(how="all")
if returns.empty:
    raise RuntimeError("No returns computed.")

period_means = returns.mean(numeric_only=True)
period_cov   = returns.cov()
period_corr  = returns.corr()

ann_factor = {"1d": 252, "1wk": 52, "1mo": 12, "3mo": 4}.get(frequency, 12)
ann_means  = period_means * ann_factor
ann_cov    = period_cov * ann_factor

core_tickers = list(prices.columns)
n = len(core_tickers)

# ---------- WEIGHTS ----------
if weights is None:
    w = np.full(n, 1.0 / n)
else:
    w = np.array([float(weights.get(t, 0.0)) for t in core_tickers], dtype=float)
    s = w.sum()
    w = np.full(n, 1.0 / n) if s == 0 else (w / s)

# ---------- PLOTS ----------
corr_png = out_dir / "correlation_matrix.png"
cov_png  = out_dir / "covariance_matrix.png"

plt.figure(figsize=(6, 5))
plt.imshow(period_corr.loc[core_tickers, core_tickers], interpolation="none")
plt.colorbar(); plt.title("Correlation Matrix")
plt.xticks(range(n), core_tickers, rotation=90)
plt.yticks(range(n), core_tickers)
plt.tight_layout(); plt.savefig(corr_png, dpi=150); plt.close()

plt.figure(figsize=(6, 5))
plt.imshow(period_cov.loc[core_tickers, core_tickers], interpolation="none")
plt.colorbar(); plt.title("Covariance Matrix")
plt.xticks(range(n), core_tickers, rotation=90)
plt.yticks(range(n), core_tickers)
plt.tight_layout(); plt.savefig(cov_png, dpi=150); plt.close()

# ---------- DASHBOARD ----------
try:
    with pd.ExcelWriter(excel_path, engine="xlsxwriter", mode="w") as writer:
        wb = writer.book
        ws_name = "Dashboard"
        ws = wb.add_worksheet(ws_name)
        writer.sheets[ws_name] = ws
        ws.activate()

        # Layout anchors
        ncols = len(core_tickers)
        PRICES_ROW, PRICES_COL = 1, 0
        RET_COL = PRICES_COL + ncols + 2
        AVG_ROW, AVG_COL = 1, RET_COL + ncols + 4
        COV_ROW, COV_COL = AVG_ROW + 5, AVG_COL
        COR_ROW, COR_COL = COV_ROW + ncols + 4, AVG_COL
        WGT_ROW, WGT_COL = 1, AVG_COL + ncols + 10
        SUMM_ROW, SUMM_COL = COR_ROW + ncols + 8, AVG_COL

        # Formats
        num  = wb.add_format({"num_format": "0.00"})
        pct  = wb.add_format({"num_format": "0.00%"})
        bold = wb.add_format({"bold": True})
        merge_format = wb.add_format({
            "align": "center",
            "valign": "vcenter",
            "bold": True,
            "border": 1
        })

        # === HEADERS (Merged + Centered) ===
        price_last_col = PRICES_COL + len(core_tickers)
        ret_last_col = RET_COL + len(core_tickers)
        ws.merge_range(PRICES_ROW - 1, PRICES_COL, PRICES_ROW - 1, price_last_col, "Prices", merge_format)
        ws.merge_range(PRICES_ROW - 1, RET_COL, PRICES_ROW - 1, ret_last_col, "Returns", merge_format)

        # === PRICES ===
        ws.write(PRICES_ROW, PRICES_COL, "Date", bold)
        for j, t in enumerate(core_tickers, start=PRICES_COL + 1):
            ws.write(PRICES_ROW, j, t, bold)
        for i, (d, row) in enumerate(prices.iterrows(), start=PRICES_ROW + 1):
            ws.write(i, PRICES_COL, d.strftime("%Y-%m-%d"))
            for j, t in enumerate(core_tickers, start=PRICES_COL + 1):
                ws.write(i, j, safe_val(row.get(t)))
        ws.set_column(PRICES_COL + 1, PRICES_COL + ncols, 12, num)

        # Table (no filters)
        price_last_row = PRICES_ROW + len(prices)
        price_range = f"${xl_col(PRICES_COL)}${PRICES_ROW + 1}:${xl_col(price_last_col)}${price_last_row}"
        ws.add_table(
            price_range,
            {
                "name": "PricesTbl",
                "columns": [{"header": h} for h in ["Date"] + core_tickers],
                "style": "Table Style Light 1",
                "autofilter": False
            }
        )

        # === RETURNS ===
        ws.write(PRICES_ROW, RET_COL, "Date", bold)
        for j, t in enumerate(core_tickers, start=RET_COL + 1):
            ws.write(PRICES_ROW, j, t, bold)
        for i, (d, row) in enumerate(returns.iterrows(), start=PRICES_ROW + 1):
            ws.write(i, RET_COL, d.strftime("%Y-%m-%d"))
            for j, t in enumerate(core_tickers, start=RET_COL + 1):
                ws.write(i, j, safe_val(row.get(t)))
        ws.set_column(RET_COL + 1, RET_COL + ncols, 12, pct)

        # Table (no filters)
        ret_last_row = PRICES_ROW + len(returns)
        ret_range = f"${xl_col(RET_COL)}${PRICES_ROW + 1}:${xl_col(ret_last_col)}${ret_last_row}"
        ws.add_table(
            ret_range,
            {
                "name": "ReturnsTbl",
                "columns": [{"header": h} for h in ["Date"] + core_tickers],
                "style": "Table Style Light 1",
                "autofilter": False
            }
        )

        # === Average Returns ===
        ws.write(AVG_ROW - 1, AVG_COL, "Average Returns", bold)
        ws.write(AVG_ROW, AVG_COL, "Per. Mean")
        ws.write_row(AVG_ROW, AVG_COL + 1, safe_row([period_means.get(t) for t in core_tickers]))
        ws.write(AVG_ROW + 1, AVG_COL, "Ann. Mean")
        ws.write_row(AVG_ROW + 1, AVG_COL + 1, safe_row([ann_means.get(t) for t in core_tickers]))
        ws.set_row(AVG_ROW, None, pct)
        ws.set_row(AVG_ROW + 1, None, pct)

        # === Rf ===
        rf_cell_row = AVG_ROW + 1
        rf_cell_col = AVG_COL + 1 + ncols + 1
        ws.write(AVG_ROW, rf_cell_col, "Rf (ann.)", bold)
        ws.write(rf_cell_row, rf_cell_col, safe_val(rf_annual))

        # === Covariance ===
        ws.write(COV_ROW - 1, COV_COL, "Covariance (annualized)", bold)
        ws.write_row(COV_ROW, COV_COL + 1, core_tickers)
        for i, t in enumerate(core_tickers, start=COV_ROW + 1):
            ws.write(i, COV_COL, t)
            ws.write_row(i, COV_COL + 1, safe_row(ann_cov.loc[t, core_tickers].values))

        # === Correlation ===
        ws.write(COR_ROW - 1, COR_COL, "Correlation", bold)
        ws.write_row(COR_ROW, COR_COL + 1, core_tickers)
        for i, t in enumerate(core_tickers, start=COR_ROW + 1):
            ws.write(i, COR_COL, t)
            ws.write_row(i, COR_COL + 1, safe_row(period_corr.loc[t, core_tickers].values))

        # === Weights ===
        ws.write(WGT_ROW - 1, WGT_COL, "Weights (edit)", bold)
        ws.write_column(WGT_ROW, WGT_COL, core_tickers)
        w_col = WGT_COL + 1
        for i, val in enumerate(w, start=WGT_ROW):
            ws.write_number(i, w_col, float(val))
        sum_row = WGT_ROW + len(core_tickers)
        ws.write(sum_row, WGT_COL, "Sum", bold)
        ws.write_formula(sum_row, w_col, f"=SUM({xl_col(w_col)}{WGT_ROW + 1}:{xl_col(w_col)}{sum_row})")
        ws.set_column(WGT_COL, WGT_COL + 1, 12, pct)

        
        avgann_start  = f"${xl_col(AVG_COL + 1)}${AVG_ROW + 2}"
        avgann_end    = f"${xl_col(AVG_COL + len(core_tickers))}${AVG_ROW + 2}"
        weights_start = f"${xl_col(w_col)}${WGT_ROW + 1}"
        weights_end   = f"${xl_col(w_col)}${WGT_ROW + len(core_tickers)}"
        cov_start     = f"${xl_col(COV_COL + 1)}${COV_ROW + 2}"
        cov_end       = f"${xl_col(COV_COL + len(core_tickers))}${COV_ROW + 1 + len(core_tickers)}"
        rf_abs        = f"${xl_col(rf_cell_col)}${rf_cell_row + 1}"

        wb.define_name("AvgAnn",  f"='{ws_name}'!{avgann_start}:{avgann_end}")
        wb.define_name("Weights", f"='{ws_name}'!{weights_start}:{weights_end}")
        wb.define_name("CovAnn",  f"='{ws_name}'!{cov_start}:{cov_end}")
        wb.define_name("Rf",      f"='{ws_name}'!{rf_abs}")

        # Summary Analysis
        ws.write(SUMM_ROW - 1, SUMM_COL, "Portfolio Summary (live formulas)", bold)

        #E(r)
        ws.write(SUMM_ROW, SUMM_COL, "E(Rp)")
        ws.write_formula(SUMM_ROW, SUMM_COL + 1, "=MMULT(TRANSPOSE(Weights), TRANSPOSE(AvgAnn))")

        # Variance
        ws.write(SUMM_ROW + 1, SUMM_COL, "Variance")
        ws.write_formula(SUMM_ROW + 1, SUMM_COL + 1, "=MMULT(MMULT(TRANSPOSE(Weights), CovAnn), Weights)")

        # Std
        ws.write(SUMM_ROW + 2, SUMM_COL, "Std")
        ws.write_formula(SUMM_ROW + 2, SUMM_COL + 1, f"=SQRT({xl_col(SUMM_COL + 1)}{SUMM_ROW + 2})")

        # Sharpe
        ws.write(SUMM_ROW + 3, SUMM_COL, "Sharpe")
        ws.write_formula(SUMM_ROW + 3, SUMM_COL + 1, f"=({xl_col(SUMM_COL + 1)}{SUMM_ROW + 1}-Rf)/{xl_col(SUMM_COL + 1)}{SUMM_ROW + 3}")

except PermissionError:
    alt_path = excel_path.with_name(f"{excel_path.stem}_new.xlsx")
    print(f"File locked/open. Close Excel and rerun. Alt path prepared: {alt_path}")

In [5]:
import os

file_path = r"C:\Users\holde\OneDrive\University of Denver\FIN4000\outputs\portfolio_dashboard.xlsx"

# Open file using your system's default Excel app
os.startfile(file_path)